<a href="https://colab.research.google.com/github/meahmadi/skyLand/blob/master/ajr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flatlib
!pip uninstall pyswisspeh
!pip install pyswisseph==2.00.00-2


     |████████████████████████████████| 5.7MB 2.6MB/s 
     |████████████████████████████████| 522kB 34.2MB/s 
  Created wheel for flatlib: filename=flatlib-0.2.1-cp36-none-any.whl size=5710818 sha256=ca2c4ad7ef45735d9e69e824057ee67b17481f641e06418ef617da3ae6990275
  Stored in directory: /root/.cache/pip/wheels/d1/8a/e2/251edf41c8ee7d20e10649a0ac85551aeb0b17a19eb3ff91ed
  Created wheel for pyswisseph: filename=pyswisseph-2.8.0.post1-cp36-cp36m-linux_x86_64.whl size=723502 sha256=318c4849c78923d0728497ab89d8e995498116de5fea6bfe98edaeb7f23a2c90
  Stored in directory: /root/.cache/pip/wheels/e0/af/3b/081d1759d2d296fcbbafc4a975a7773063de6ec4f5f667a10d
Successfully built flatlib pyswisseph
     |████████████████████████████████| 460kB 3.5MB/s 
  Created wheel for pyswisseph: filename=pyswisseph-2.0.0.post2-cp36-cp36m-linux_x86_64.whl size=606703 sha256=8b76339d953110946df3b4bb6468afb448133a2acec20a8145008460b198155c
  Stored in directory: /root/.cache/pip/wheels/f1/9a/ea/e9d1a3b6b38e20440c9

In [49]:
import math 

deg_abs_dist = lambda first,second: 180.0 - math.fabs(math.fmod(math.fabs(first - second), 360.0) - 180.0)
deg_sign = lambda first,second: 1 if math.fmod(deg_abs_dist(first,second) + first,360) == math.fmod(second,360) else -1 
deg_dist = lambda first,second: deg_sign(first,second) * deg_abs_dist(first,second)

def condition_moon_deg(sun,moon):
    dist = math.ceil(deg_dist(sun,moon)/360 * 29.5)%29
    return 29 if dist == 0 else dist


In [50]:
from flatlib import const
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
#!pip install pyyaml
import yaml, codecs
from flatlib import aspects
import itertools
import math
import pandas as pd
# to fix the installation bug
# pip uninstall pyswisspeh
# pip install pyswisseph==2.00.00-2
# Documentation
# https://flatlib.readthedocs.io/en/latest/tutorials/chart-properties.html


class Zodiac(object):
    STARS = ['Alpheratz', 'Algol', 'Alcyone', 'Alcyone', 'Aldebaran', 'Rigel', 'Capella', 'Betelgeuse', 'Sirius', 'Canopus', 'Castor', 'Pollux', 'Procyon', 'Asellus_Borealis', 'Asellus_Australis', 'Alphard', 'Regulus', 'Denebola', 'Algorab', 'Spica', 'Arcturus', 'Alphecca', 'ZUBEN_ELSCHEMALI', 'Unukalhai', 'Agena', 'RIGEL_CENTAURUS', 'Antares', 'Lesath', 'Vega', 'Altair', 'Deneb_Algedi', 'Fomalhaut', 'DENEB_ADIGE', 'Achernar','Algenib']
    PLANETS = ['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn', 'North_Node', 'South_Node', 'Syzygy', 'Pars_Fortuna']
    all_objects = []
    all_stars = []    
    all_planets = []    
    
    for star in STARS:
        all_stars.append(eval(F"const.STAR_{star.upper()}"))

    for planet in PLANETS:
        all_planets.append(eval(F"const.{planet.upper()}"))
    
    all_objects = all_stars + all_planets
    
    def __init__(self):
        self._parse_signs()
        self._parse_planets()
    
    @staticmethod
    def check_aspects(hrscp):
        major_aspects = []
        minor_aspects = []
        
        for a in Zodiac.all_objects:
            for b in Zodiac.all_objects:
                if a!=b:
                    try:
                        A = hrscp.chart.get(a)
                    except:
                        A = hrscp.chart.getFixedStar(a)
                    try:
                        B = hrscp.chart.get(b)
                    except:
                        B = hrscp.chart.getFixedStar(b)

                    try:
                        aspect = aspects.getAspect(A, B, const.MAJOR_ASPECTS)
                        if aspect.type!=-1:
                            res = ([aspect.active.id, aspect.passive.id], aspect.type,aspect.movement())
                            if res not in major_aspects:
                                major_aspects.append(res)
                    except:
                        pass
                    
                    try:
                        aspect = aspects.getAspect(A, B, const.MINOR_ASPECTS)
                        if aspect.type!=-1:
                            res = ([aspect.active.id, aspect.passive.id], aspect.type,aspect.movement())
                            if res not in major_aspects:
                                minor_aspects.append(res)
                    except:
                        pass
        return {'major_aspects':major_aspects,'minor_aspects': minor_aspects}
    
    def _parse_signs(self):
        f = codecs.open('sign_conditions.yaml','r','utf-8')
        self.signs = yaml.safe_load(f)
        
    def _parse_planets(self):
        f = codecs.open('condition_planets.yml','r','utf-8')
        self.planets = yaml.safe_load(f)
        

class Horoscope(object):
    
    def __init__(self, data='2015/03/13', time='17:00', time_zone='+00:00', geo_lat = '38n32',geo_long= '8w54'):
        self.date = Datetime(data, time, time_zone)
        self.pos = GeoPos(geo_lat, geo_long)
        self.chart = Chart(self.date, self.pos, hsys=const.HOUSES_PLACIDUS)    

    def get_obj(self, name):
        return self.chart.getObject(eval(F"const.{name.upper()}")).__dict__
        
    def get_mars(self):
        return self.chart.getObject(const.MARS).__dict__

    def get_mercury(self):
        return self.chart.getObject(const.MERCURY).__dict__    
    
    def get_venus(self):
        return self.chart.getObject(const.VENUS).__dict__ 

    def get_spica(self):
        return self.chart.getFixedStar('Spica').__dict__
    
    def get_sun(self):
        return self.chart.getObject(const.SUN).__dict__

    def get_moon(self):
        return self.chart.getObject(const.MOON).__dict__

    def get_stars(self):
        '''
            get all stars
        '''        
        stars = dict()
        for idx, star in enumerate(Zodiac.all_stars):
            temp = self.chart.getFixedStar(star).__dict__
            stars[F"{temp['id']}"] = temp

        return stars
    
    def get_houses(self):
        '''
            get all houses
        '''        
        houses = dict()
        for idx, house in enumerate([const.HOUSE1,const.HOUSE2,const.HOUSE2,const.HOUSE4,const.HOUSE5,const.HOUSE6,const.HOUSE7,const.HOUSE8,const.HOUSE9,const.HOUSE10,const.HOUSE11,const.HOUSE12]):
            houses[F'HOUSE_{idx+1}'] = self.chart.get(house).__dict__
            try:
                houses[F'HOUSE_{idx+1}']['condition'] = self.chart.get(house).condition()
            except:
                pass
            try:
                houses[F'HOUSE_{idx+1}']['gender'] = self.chart.get(house).gender()            
            except:
                pass
            
             
            houses[F'HOUSE_{idx+1}']['objects'] = []
            for obj in Zodiac.all_objects:
                try:
                    obj = self.chart.get(obj)
                except:
                    obj = self.chart.getFixedStar(obj)
                if self.chart.get(house).hasObject(obj):                    
                    houses[F'HOUSE_{idx+1}']['objects'].append(obj.id) 

        return houses
    
    def get_objects(self):
        '''
            get all objects
        '''
        objects = dict()
        for obj in self.chart.objects:
            objects[(obj.id)] = obj.__dict__
        return objects
        

    def get_angles(self):
        '''
            get all angles
        '''
        objects = dict()
        for obj in self.chart.angles:
            objects[(obj.id)] = obj.__dict__
        return objects
                
    
    def get_chart_properties(self):
        '''
            get chart properties
        '''
        return {'isDiurnal': self.chart.isDiurnal(),'MoonPhase':self.chart.getMoonPhase()}
    
    def get_aspects(self):
        return Zodiac.check_aspects(self)

import tqdm
from collections import OrderedDict
from datetime import datetime, timedelta

def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta 
        
from math import sin, cos, sqrt, atan2, radians
import numpy as np


def measure_distance(obj1, obj2):

    R = 1

    lat1 = radians(obj1['lat'])
    lon1 = radians(obj1['lon'])
    lat2 = radians(obj2['lat'])
    lon2 = radians(obj2['lon'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# def condition_moon(sun, moon):
#     dist = (-sun['lon'] + moon['lon']+360)%360
#     return math.ceil((dist/360)*29.5)+2   
def condition_moon(sun,moon):
    return condition_moon_deg(sun['lon'],moon['lon'])


def condition_observable_rise(sun, mars):
    return deg_dist(mars['lon'],sun['lon']) > 4.5 and deg_dist(mars['lon'],sun['lon']) < 5

def condition_in_virgo(spica, mars, o1, o2):
    conditions = []
    for o in [mars, o1, o2]:
        # c1 = spica['lon'] < (o['lon']+35)
        # c2 = spica['lon'] > (o['lon']-15)
        c1 = deg_dist(o['lon'],spica['lon']) < 35
        c2 = deg_dist(spica['lon'],o['lon']) < 15
        c3 = o['lat'] > -10
        c4 = o['lat'] < 20
        conditions.append(all([c1,c2,c3,c4]))
    # return all(conditions + [mars['lon'] > o1['lon'], mars['lon'] > o2['lon']])
    return all(conditions + [deg_sign(o1['lon'],mars['lon'])>0 , deg_sign(o2['lon'],mars['lon'])>0 ])
    

In [59]:
hr = Horoscope(data='2020/10/18',time='02:12',time_zone='+02:56', geo_lat='32n60', geo_long='44e02')
moon = hr.get_moon()
sun = hr.get_sun()
print(moon)
print(sun)
print(math.ceil(deg_dist(sun['lon'],moon['lon'])/360 * 29.5)%29)

{'id': 'Moon', 'type': 'Planet', 'lon': 221.5574449718306, 'lat': 3.2697168211580974, 'sign': 'Scorpio', 'signlon': 11.557444971830591, 'lonspeed': 15.196993772717924, 'latspeed': -1.0360543579791064} {'id': 'Sun', 'type': 'Planet', 'lon': 205.03918136004376, 'lat': 1.5626928916493712e-05, 'sign': 'Libra', 'signlon': 25.039181360043756, 'lonspeed': 0.9931613102487282, 'latspeed': -2.2401534437023653e-05}
2


In [62]:
#cities = {'Karbala':('32n60','44e02'),'San Fransisco':('37n78','122w42'),'Buenos Aires':('34s60','58w38'), 'Stockholm': ('59n33','18e07') ,'Tehran':('35n69','51e39'),'Bloemfontein':('29s09', '26e16'),'Beijing' :('39n90', '116e41'),'Melbourne': ('37s81', '144e96')}

cities = {'Karbala':('32n60','44e02')}

#dts = [(dt.strftime('%Y/%m/%d'),dt.strftime('%H:%M')) for dt in datetime_range(datetime(-2000, 1, 1), datetime(2032, 12, 1), timedelta(minutes=60*24))]
dts = [(dt.strftime('%Y/%m/%d'),dt.strftime('%H:%M')) for dt in datetime_range(datetime(1, 1, 1), datetime(1, 12, 31), timedelta(minutes=60*24))]
objects_oi = ['Mercury', 'Venus', 'Jupiter', 'Saturn']

time_zone='+02:56'
results = {'date':[],'city':[],'time':[],'month':[], 'o1':[],'o2':[],'Mars<Spica':[]}

for city, (geo_lat, geo_long) in cities.items():
    for y in tqdm.tqdm(range(-3000,2000)):
        for d,t in dts:
            d = '/'.join([str(y)]+d.split('/')[1:3])
            hr = Horoscope(data=d,time='05:30',time_zone=time_zone, geo_lat=geo_lat, geo_long=geo_long)
            mars = hr.get_mars()
            sun = hr.get_sun()
            if condition_observable_rise(sun, mars):
                for o1_name, o2_name in list(itertools.combinations(objects_oi, 2)):
                    o1=hr.get_obj(o1_name)
                    o2=hr.get_obj(o2_name)
                    spica = hr.get_spica()    
                    if condition_in_virgo(spica, mars, o1, o2):
                        moon = hr.get_moon()
                        results['city'].append(city)
                        results['date'].append(d)
                        results['time'].append('05:30')
                        results['month'].append(condition_moon(sun, moon))
                        results['o1'].append(o1_name)            
                        results['o2'].append(o2_name)   
                        results['Mars<Spica'].append(deg_sign(mars['lon'] , spica['lon']))

100%|██████████| 5000/5000 [27:42<00:00,  3.01it/s]


In [63]:
df = pd.DataFrame(results)

In [64]:
pd.DataFrame(results).to_csv('all.csv')
#olddf = df
#oldresults = results

In [73]:
df

,date,city,time,month,o1,o2,Mars<Spica
0,-2965/08/27,Karbala,05:30,12,Venus,Jupiter,-1
1,-2965/08/28,Karbala,05:30,13,Venus,Jupiter,-1
2,-2918/08/19,Karbala,05:30,14,Venus,Jupiter,-1
3,-2918/08/20,Karbala,05:30,15,Venus,Jupiter,-1
4,-2888/07/20,Karbala,05:30,15,Mercury,Jupiter,1
...,...,...,...,...,...,...,...
132,1878/10/03,Karbala,05:30,8,Mercury,Venus,1
133,1910/10/12,Karbala,05:30,8,Mercury,Venus,1
134,1942/10/20,Karbala,05:30,11,Mercury,Venus,1
135,1942/10/21,Karbala,05:30,12,Mercury,Venus,1


In [76]:
df[(df['month']>7)&(df['month']<12)&(df['Mars<Spica']==-1)]

,date,city,time,month,o1,o2,Mars<Spica
32,-2602/09/01,Karbala,05:30,8,Mercury,Venus,-1
33,-2602/09/02,Karbala,05:30,9,Mercury,Venus,-1
37,-2555/08/25,Karbala,05:30,11,Mercury,Saturn,-1
91,-535/09/25,Karbala,05:30,10,Venus,Jupiter,-1
115,680/10/10,Karbala,05:30,8,Mercury,Venus,-1
